# eventchain

> Building and anylizing event chains in a convenient way.

In [ ]:
#| default_exp core

In [ ]:
#| export
import pandas as pd
from scipy import stats
from scipy.stats import norm

In [ ]:
#| export

class EventChain:
    
    def __init__(self, df1: pd.DataFrame, df2: pd.DataFrame, user_col: str, timestamp_col: str, suffix):
        
        self.timestamp_a = f"{timestamp_col}_{suffix[0]}"
        self.timestamp_b = f"{timestamp_col}_{suffix[1]}"

        self.user_col = user_col    
        self.any_event_A = df1.rename(columns={timestamp_col: self.timestamp_a})
        self.any_event_B = df2.rename(columns={timestamp_col: self.timestamp_b})
        self.first_event_A = self.any_event_A.groupby(self.user_col)[f"{timestamp_col}_{suffix[0]}"].min().reset_index()
        self.last_event_A = self.any_event_A.groupby(self.user_col)[f"{timestamp_col}_{suffix[0]}"].max().reset_index()
        self.first_event_B = self.any_event_B.groupby(self.user_col)[f"{timestamp_col}_{suffix[1]}"].min().reset_index()
        self.last_event_B = self.any_event_B.groupby(self.user_col)[f"{timestamp_col}_{suffix[1]}"].max().reset_index()
            
    def anti_join(self, df1, df2): 
        return df1.merge(df2, how="left", on=self.user_col, indicator=True).query("_merge == 'left_only'").drop(columns="_merge")

    def before(self, df1, df2): 
        return (df1
            .merge(df2, how="inner", on=self.user_col).query(f"{self.timestamp_a} < {self.timestamp_b}")
            .assign(gap=lambda x: x[self.timestamp_b] - x[self.timestamp_a]))

    def before_or_equal(self, df1, df2): 
        return (df1
            .merge(df2, how="inner", on=self.user_col).query(f"{self.timestamp_a} <= {self.timestamp_b}")
            .assign(gap=lambda x: x[self.timestamp_b] - x[self.timestamp_a]))

    @property
    def first_before_first(self): 
        return self.before(self.first_event_A, self.first_event_B)

    @property
    def first_before_or_equal_first(self):
        return self.before_or_equal(self.first_event_A, self.first_event_B)
    
    @property
    def first_before_last(self): 
        return self.before(self.first_event_A, self.last_event_B)

    @property
    def first_before_or_equal_last(self): 
        return self.before_or_equal(self.first_event_A, self.last_event_B)
    
    @property
    def last_before_first(self):
        return self.before(self.last_event_A, self.first_event_B)

    @property
    def last_before_or_equal_first(self):
        return self.before_or_equal(self.last_event_A, self.first_event_B)

    @property
    def last_before_last(self):
        return self.before(self.last_event_A, self.last_event_B)

    @property
    def last_before_or_equal_last(self):
        return self.before_or_equal(self.last_event_A, self.last_event_B)

    @property
    def any_before_any(self):
        return self.before_or_equal(self.any_event_A, self.any_event_B)

    @property
    def any_before_first(self):
        return self.before_or_equal(self.any_event_A, self.first_event_B)

    @property
    def any_before_or_equal_first(self):
        return self.before_or_equal(self.any_event_A, self.first_event_B)

    @property
    def any_before_last(self):
        return self.before_or_equal(self.any_event_A, self.last_event_B)

    @property
    def any_before_or_equal_last(self):
        return self.before_or_equal(self.any_event_A, self.last_event_B)

    @property
    def first_before_none(self): 
        return self.anti_join(self.first_event_A, self.any_event_B)
    
    @property
    def last_before_none(self): 
        return self.anti_join(self.last_event_A, self.any_event_B)
    
    @property
    def none_before_first(self): 
        return self.anti_join(self.first_event_B, self.any_event_A)
    
    @property
    def none_before_last(self): 
        return self.anti_join(self.last_event_B, self.any_event_A)

In [ ]:
#| export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
# write test class
def ClassTest():
    assert say_hello("Jeremy") == "Hello Jeremy!"

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()